In [6]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as f 
import torchvision
import torchvision.datasets as datasets 
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader 
from torch.utils.tensorboard import SummaryWriter

class Discriminator(nn.Module): 
    def __init__(self, img_dim): 
        super().__init__() 
        self.disc = nn.Sequential(
            nn.Linear(img_dim, 128),
            nn.LeakyReLU(0.1),
            nn.Linear(128,1),
            nn.Sigmoid(),
        )
        
    def forward(self, x):
        return self.disc(x)


class Generator(nn.Module): 
    def __init__(self, z_dim, img_dim): 
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, img_dim),
            nn.Tanh(),
        )

    def forward(self, x):
        return self.gen(x)
    
# Hyperparameters etc. 

device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4
z_dim = 64 
image_dim = 28 * 28 * 1
batch_size = 32 
num_epochs = 50 

disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, ), (0.5), )])

dataset = datasets.MNIST(root="../dataset/", transform=transforms, download=False)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()
### torch.nn.BCELoss 
# L(x,y) = {Li, ... , Ln}^T
# Ln = -wn[yn(log(xn)) + (1 -yn)log(1-xn)]
# Mean(L(xx,y))
# sum(L(xx,y))

writer_fake = SummaryWriter(f"runs/GAN_MINIST/fake")
writer_real = SummaryWriter(f"runs/GAN_MINIST/real")
step = 0 

for epoch in range(num_epochs): 
    for batch_idx, (real, _) in enumerate(loader): 
        real = real.view(-1, 784).to(device)
        batch_size = real.shape[0]
        
        ### Traning Discriminator : max log(D(real)) + log(1- D(G(z)))
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).view(-1)
        lossD_fake = criterion(disc_fake, torch.ones_like(disc_fake))
        lossD = (lossD_real + lossD_fake) / 2 
        disc.zero_grad() 
        lossD.backward(retain_graph=True)
        opt_disc.step()
        
        ### Train Generator min log(1 - D(G(z))) <-> max log(D(G(z)))
        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        lossG.backward() 
        opt_gen.step() 
        
        if batch_idx == 0: 
            print(f" Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} Loss D: {lossD:.8f}, Loss G : {lossG:.8f}")
            
            with torch.no_grad(): 
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize = True)
                img_grid_real = torchvision.utils.make_grid(data, normalize = True)
                
                writer_fake.add_image("MNIST Fake Images", img_grid_fake, global_step=step)
                writer_fake.add_image("MNIST Real Images", img_grid_real, global_step=step)
                
                step += 1

#         https://www.youtube.com/watch?v=OljTVUVzPpM&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=24    



'''
Things to try : 
    1. What happens if you use larger network? 
    2. Better normalization with BatchNorm 
    3. Different learning rate 
    4. Change architenture to CNN
'''

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_34988/1509590845.py", line 5, in <cell line: 5>
    import torchvision
  File "/opt/conda/lib/python3.8/site-packages/torchvision/__init__.py", line 6, in <module>
    from torchvision import models
  File "/opt/conda/lib/python3.8/site-packages/torchvision/models/__init__.py", line 12, in <module>
    from . import detection
  File "/opt/conda/lib/python3.8/site-packages/torchvision/models/detection/__init__.py", line 1, in <module>
    from .faster_rcnn import *
  File "/opt/conda/lib/python3.8/site-packages/torchvision/models/detection/faster_rcnn.py", line 4, in <module>
    from torchvision.ops import MultiScaleRoIAlign
  File "/opt/conda/lib/python3.8/site-packages/torchvision/ops/__init__.py", line 14, in <module>
    _register_custom_op()
  File "/opt/conda/lib/pyt

In [2]:
real = real.view(-1, 784).to(device)
batch_size = real.shape[0]
        

In [3]:
print(real)

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]], device='cuda:0')


In [4]:
print(batch_size)

32


In [3]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as f 
import torchvision
import torchvision.datasets as datasets 
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader 
from torch.utils.tensorboard import SummaryWriter

class Discriminator(nn.Module): 
    def __init__(self, img_dim): 
        super().__init__() 
        self.disc = nn.Sequential(
            nn.Linear(img_dim, 128),
            nn.LeakyReLU(0.1),
            nn.Linear(128,1),
            nn.Sigmoid(),
        )
        
    def forward(self, x):
        return self.disc(x)


class Generator(nn.Module): 
    def __init__(self, z_dim, img_dim): 
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, img_dim),
            nn.Tanh(),
        )

    def forward(self, x):
        return self.gen(x)
    
# Hyperparameters etc. 

device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4
z_dim = 64 
image_dim = 28 * 28 * 1
batch_size = 32 
num_epochs = 50 

disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, ), (0.5), )])     # Composes several transforms together.

dataset = datasets.MNIST(root="../dataset/", transform=transforms, download=False)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()
### torch.nn.BCELoss 
# L(x,y) = {Li, ... , Ln}^T
# Ln = -wn[yn(log(xn)) + (1 -yn)log(1-xn)]
# Mean(L(xx,y))
# sum(L(xx,y))

writer_fake = SummaryWriter(f"runs/GAN_MINIST/fake")
writer_real = SummaryWriter(f"runs/GAN_MINIST/real")
step = 0 

for epoch in range(num_epochs): 
    for batch_idx, (real, _) in enumerate(loader): 
        real = real.view(-1, 784).to(device)
        batch_size = real.shape[0]
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        
        ### Traning Discriminator : max log(D(real)) + log(1- D(G(z)))
        disc_real = disc(real).view(-1)
        disc_fake = disc(fake).view(-1)
            
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        lossD_fake = criterion(disc_fake, torch.ones_like(disc_fake))
        
        lossD = (lossD_real + lossD_fake) / 2 
        disc.zero_grad() 
        lossD.backward(retain_graph=True)
        opt_disc.step()
        
        ### Train Generator min log(1 - D(G(z))) <-> max log(D(G(z)))
        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step() 
        
        if batch_idx == 0: 
            print(f" Epoch [{epoch}/{num_epochs}] Loss D: {lossD:.8f}, Loss G : {lossG:.8f}")
            
            with torch.no_grad(): 
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize = True)
                img_grid_real = torchvision.utils.make_grid(data, normalize = True)
                
                writer_fake.add_image("MNIST Fake Images", img_grid_fake, global_step=step)
                writer_real.add_image("MNIST Real Images", img_grid_real, global_step=step)
                
                step += 1

#         https://www.youtube.com/watch?v=OljTVUVzPpM&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=24    



'''
Things to try : 
    1. What happens if you use larger network? 
    2. Better normalization with BatchNorm 
    3. Different learning rate 
    4. Change architenture to CNN
'''

ModuleNotFoundError: No module named 'torch.fx'

In [ ]:
pip install torch==1.9.0

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |███                             | 75.2 MB 109 kB/s eta 1:55:13